# Design Patterns in Python (pycon 2019)
Idioms (Language specific) - 
Architecture Patterns (MVC etc) - 
Design Patterns (GoF)

---
### Design Patterns Classified:
- Creational Pattern
- Structural Pattern
- Behavioral Pattern


### Design Principles:
A- Separate out the things that change from those that stay the same.
B- Program to an interface, not an implementation. (Take a way information that are not important for the user; Use private method and fields when you can)
C- Prefer composition and delegation over inheritance.

### Anatomy of Design Patterns:
intent 
Motivation
Structure (UML)
Implementation

----


## Singleton Pattern
It’s a creational design pattern that lets you ensure that a class has just one instance, while creating a global access point to this instance. Example: Connection to server. You need just one connection and not many at once.

In [49]:
class _Tigger:
    def __str__(self):
        return "I'm the only one!"
    
    def roar(self):
        return "grrr!"

# This part of the code takes care of single creation of the Tigger class.
_instance = None
def Tigger():
    global _instance
    if _instance is None:
        _instance = _Tigger()
    return _instance

In [50]:
a = Tigger()
b = Tigger()

print(f'ID(a) = {id(a)}')
print(f'ID(b) = {id(b)}')
print(f'Are a and b the same object? {a is b}')

ID(a) = 4498646128
ID(b) = 4498646128
Are a and b the same object? True


---
## Template Method Patterns
It's a behaviour pattern that define a skeleton of an Algorithm in the base class but lets derived classes override specific steps of the algorithm without changing its structure.
this pattern suggests that you break down an algorithm into series of steps, turn this steps into methods, and put series of calls to this methods inside a single 'template method'.


In [51]:
from abc import ABC, abstractmethod

class AverageCalculator(ABC):  #1
    def average(self):  #2
        try:
            num_items = 0
            total_sum = 0
            while self.has_next():
                total_sum += self.next_item()
                num_items += 1
            if num_items == 0:
                raise RuntimeError("Can't compute the average of zero items")
            return total_sum/num_items
        finally:
            self.dispose()
    
    @abstractmethod  # 3
    def has_next(self):
        pass
    
    @abstractmethod
    def next_item(self):
        pass
    
    # this has a default functionality
    def dispose(self):
        pass

In [52]:
class FileAverageCalculator(AverageCalculator):  # 4
    def __init__(self, file):
        self.file = file
        self.last_line = self.file.readline()
        
    def has_next(self):  # 5
        return self.last_line != ''
    
    def next_item(self):
        result = float(self.last_line)
        self.last_line = self.file.readline()
        return result
    
    def dispose(self):
        self.file.close()  # call the template method

In [53]:
fac = FileAverageCalculator(open('numbers.txt'))
print(f'Average is: {fac.average()}')

Average is: 7.0


In [54]:
class MemoryAverageCalculator(AverageCalculator):  # 6
    def __init__(self, numbers):
        self.numbers = numbers
        self.index = 0
        
    def has_next(self):  # 7
        return len(self.numbers) > self.index
    
    def next_item(self):
        result = self.numbers[self.index]
        self.index += 1
        return result

In [55]:
mac = MemoryAverageCalculator([1, 2, 3, 4, 5, 6, 7, 8, 9])
print(f'Avarage is: {mac.average()}')

Avarage is: 5.0


---
## Adapter
A structure design pattern that converts the interface of a class into another interface client expected. Adapter lets classes work together that couldn't otherwise Because of Incompatible Interfaces.

In [59]:
class GeneratorAdapter:
    def __init__(self, adaptee):
        self.adaptee = adaptee
        
    def readline(self):
        try:
            return next(self.adaptee)
        except StopIteration:
            return ''
        
    def close(self):
        pass
    
from random import randint
generator = (randint(1,100) for i in range(100000))
fac = FileAverageCalculator(GeneratorAdapter(generator))  
"""
as you see above, instead of giving a file, we give it a generator.
The FileAverageCalculator is getting a file as input, but our new class, GeneratorAdapter
helps us to adapte a generator instead of a file.
"""
print(fac.average())

50.41486


----
second example:

In [61]:
class Duck:
    def quack(self):
        print('Quack Quack')
        
    def fly(self):
        print('I am flying')
        

class Turkey:
    def gobble(self):
        print('gobble gobble')
        
    def fly(self):
        print('I am flying a short distance')

In [62]:
class TurkeyAdapter:
    '''
    We need an adapter which get's an input (bird) and returns two methods from the Turkey class.
    Differences are that no matter what the input bird is, it will gobble and it will fly longer.
    in this case we just print the fly() 5 times.
    '''
    def __init__(self, adaptee):
        self.adaptee = adaptee
        
    def quack(self):
        self.adaptee.gobble()
    
    def fly(self):
        for i in range(5):
            self.adaptee.fly()

In [73]:
def duck_interaction(duck):
    duck.quack()
    duck.fly()
    
def turkey_interaction(turkey):
    turkey.gobble()
    turkey.fly()
    
duck = Duck()
turkey = Turkey()
turkey_adapter = TurkeyAdapter(turkey)

print('The turkey says...')
turkey_interaction(turkey)

print('\nThe duck says...')
duck_interaction(duck)

print('\nThe turkey_adapter says...')
duck_interaction(turkey_adapter)

The turkey says...
gobble gobble
I am flying a short distance

The duck says...
Quack Quack
I am flying

The turkey_adapter says...
gobble gobble
I am flying a short distance
I am flying a short distance
I am flying a short distance
I am flying a short distance
I am flying a short distance


----
## Observer
It's a behaviour design pattern that defines a one to many dependency between objects so that when one object changes state, all its dependents are Notified and update aromatically.

In [75]:
from abc import ABC, abstractmethod

class Observer(ABC):
    '''
    This is an abstract class.
    Subscribers 
    '''
    @abstractmethod
    def update(self, Observable, *args):
        pass
    

class Observable:
    '''
    Publisher
    '''
    def __init__(self):
        self.__observers = []
        
    def add_observer(self, observer):
        self.__observers.append(observer)
        
    def delete_observer(self, observer):
        self.__observers.remove(observer)
        
    def notify_observers(self, *args):
        for observer in self.__observers:
            observer.update(self, *args)

In [84]:
# Observable or publisher
class Employee(Observable):
    
    def __init__(self, name, salary):
        super().__init__()
        self._name = name
        self._salary = salary
        
    @property
    def name(self):
        return self._name
    
    @property
    def salary(self):
        return self._salary
    
    @salary.setter
    def salary(self, new_salary):
        self._salary = new_salary
        self.notify_observers(new_salary)
        

# observers or subscribers classes are Payroll and TaxMan
class Payroll(Observer):
    
    def update(self, changed_employee, new_salary):
        print(f'Cut a new check for {changed_employee.name}. '
        f'Her/His new salary is {new_salary}.')
        
        
class TaxMan(Observer):
    
    def update(self, changed_employee, new_salary):
        print(f'Send {changed_employee.name} a new tax bill.')

In [85]:
e = Employee('Sam Levin', 100000)
p = Payroll()
t = TaxMan()

# Subscribing 
e.add_observer(p)
e.add_observer(t)

# Updating (publishing some new info)
print('Update one')
e.salary = 110000

# Unsubscribe
e.delete_observer(t)

print('\nUpdate two')
e.salary = 120000

Update one
Cut a new check for Sam Levin. Her/His new salary is 110000.
Send Sam Levin a new tax bill.

Update two
Cut a new check for Sam Levin. Her/His new salary is 120000.
